In [4]:
from traning_classes import DataManager, ModelPerformanceIndicators
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
model_names = [
    "I60-R5",
    "I60-R20",
    "I60-R60"
]

from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform  # Xavier initializer

def I60_architecture(input_shape=(96, 180)):
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(64, (5, 3), padding='same', kernel_initializer=GlorotUniform(),
                      dilation_rate=(2, 1), input_shape=(*input_shape, 1)),  # Dilation only
        layers.BatchNormalization(),  # Batch normalization
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((3, 1)),  # Simulating vertical strides

        # Second Convolutional Block
        layers.Conv2D(128, (5, 3), padding='same', kernel_initializer=GlorotUniform(),
                      strides=(1, 1), dilation_rate=(1, 1)),  # Regular convolution
        layers.BatchNormalization(),  # Batch normalization
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((2, 1)),  # Pooling with size (2, 1)

        # Third Convolutional Block
        layers.Conv2D(256, (5, 3), padding='same', kernel_initializer=GlorotUniform(),
                      strides=(1, 1), dilation_rate=(1, 1)),  # Regular convolution
        layers.BatchNormalization(),  # Batch normalization
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((2, 1)),  # Pooling with size (2, 1)

        # Third Convolutional Block
        layers.Conv2D(512, (5, 3), padding='same', kernel_initializer=GlorotUniform(),
                      strides=(1, 1), dilation_rate=(1, 1)),  # Regular convolution
        layers.BatchNormalization(),  # Batch normalization
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((2, 1)),  # Pooling with size (2, 1)

        # Flatten and Fully Connected Layers
        layers.Flatten(),
        layers.Dense(1, kernel_initializer=GlorotUniform()),  # Fully connected layer
        layers.Dropout(0.5),  # 50% dropout
        layers.Activation('sigmoid')  # Binary classification (True/False)
    ])

    # Use Adam optimizer with a learning rate of 0.01
    optimizer = Adam(learning_rate=0.01)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [6]:
import gc
import tensorflow as tf

def train_pipeline(model_name):
    try:
        # === GET DATA ===
        dm = DataManager()
        image_dir = f"../_images/{model_name}/traning"
        dm.load_images(image_dir)
        X_train, X_test, y_train, y_test = dm.split_data()

        # === GET MODEL ARCHITECTURE ===
        model = I60_architecture()

        # === TRAINING ===
        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=10,
            restore_best_weights=True
        )
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=2,
            min_lr=1e-6
        )
        callbacks = [early_stopping, reduce_lr]

        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=128,
            epochs=40,
            validation_split=0.3,
            callbacks=callbacks,
            verbose=2
        ).history

        # === EVALUATE PERFORMANCE ===
        test_predictions = model.predict(X_test, verbose=0)
        ModelPerformanceIndicators.save_pdf(test_predictions, y_test, model_name)

        # === SAVE MODEL ===
        model_filename = f'../models/{model_name}.keras'
        model.save(model_filename)

    finally:
        # === CLEAN UP MEMORY ===
        del X_train, X_test, y_train, y_test
        del history, test_predictions
        del model
        gc.collect()  # Python garbage collection
        tf.keras.backend.clear_session()  # Clears TensorFlow session/graph

In [ ]:
for model_name in model_names:
    print(f"Training model: {model_name}")
    train_pipeline(model_name)

Training model: I60-R5
TRUE: 121686 images with shape (96, 180)
FALSE: 99402 images with shape (96, 180)
Balanced counts: {0: 99402, 1: 99402}
Training set: 139162 samples
Test set: 59642 samples
Image shape: (96, 180)
Epoch 1/40
